# 02 · Construir Dataset Maestro (Unidad = Obra)

Unifica **Obra + Empresa + Miembro** y usa **Matriz de Priorización** para etiquetar:
- `PROYECTO_RIESGO` (numérico)
- `PROYECTO_RIESGO_DESC` (categórico)

Genera `data/processed/dataset_obras.parquet`.

In [15]:
from pathlib import Path
import pandas as pd
import numpy as np
import glob

# Detecta BASE como dos niveles arriba del directorio actual (asumiendo /notebooks/)
BASE = Path.cwd().parents[0]
DATA_EXT = BASE / 'data' / 'external'
DATA_PROC = BASE / 'data' / 'processed'
DATA_PROC.mkdir(parents=True, exist_ok=True)
BASE, DATA_EXT, DATA_PROC

(WindowsPath('c:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion'),
 WindowsPath('c:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion/data/external'),
 WindowsPath('c:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion/data/processed'))

## 1) Configuración (ajusta si cambian nombres)

In [23]:
CONFIG = {
  'obra_glob':        str(DATA_EXT / 'obra' / 'DS_DASH_Obra_*.csv'),
  'empresa_glob':     str(DATA_EXT / 'empresa' / 'DS_DASH_Empresa_*.csv'),
  'miembro_glob':     str(DATA_EXT / 'funcionario' / 'DS_DASH_Miembro_*.csv'),
  'prio_glob':        str(DATA_EXT / 'priorizacion' / 'DS_DASH_MatrizPriorizacion_*.csv'),
  # Claves oficiales provistas por el usuario
  'key_obra':   'codigo_unico',
  'key_ruc':    'codigo_ruc',
  'key_dni':    'codigo_dni',
  'key_prio':   'codigo_unico',  
  # Targets en Matriz Priorización
  'prio_num':   'PROYECTO_RIESGO',
  'prio_desc':  'PROYECTO_RIESGO_DESC',
  # Salida
  'output_parquet': str(DATA_PROC / 'dataset_obras.parquet')
}

## 2) Utilitarios de lectura

In [24]:
def read_csv_any(path):
    last_err = None
    for sep in [',',';','\t','|']:
        for enc in ['utf-8','latin-1','cp1252']:
            try:
                return pd.read_csv(path, sep=sep, encoding=enc), enc, sep
            except Exception as e:
                last_err = str(e)
    raise RuntimeError(f'No se pudo leer {path}: {last_err}')

def cargar_por_glob(patron: str) -> pd.DataFrame:
    paths = glob.glob(patron)
    tablas = []
    for p in paths:
        try:
            df, enc, sep = read_csv_any(p)
            tablas.append(df)
            print(f'Cargado: {Path(p).name} -> {df.shape} (enc={enc}, sep={sep})')
        except Exception as e:
            print('ERROR leyendo', p, e)
    return pd.concat(tablas, axis=0, ignore_index=True) if tablas else pd.DataFrame()

## 3) Cargar datasets por entidad (concat por patrón)

In [51]:
# === 3) Cargar datasets por entidad (concat por patrón) — VERSIÓN UNIFICADA ===
import re, glob
import numpy as np
import pandas as pd
from pathlib import Path

def read_csv_any(path):
    last_err = None
    for sep in [',',';','\t','|']:
        for enc in ['utf-8','latin-1','cp1252']:
            try:
                df = pd.read_csv(path, sep=sep, encoding=enc)
                return df, enc, sep
            except Exception as e:
                last_err = e
    raise RuntimeError(f"No se pudo leer {path}: {last_err}")

def cargar_por_glob(patron: str) -> pd.DataFrame:
    files = glob.glob(patron)
    print(f"Patrón: {patron} -> {len(files)} archivos")
    tablas = []
    for p in files:
        try:
            df, enc, sep = read_csv_any(p)
            tablas.append(df)
            print(f"Cargado: {Path(p).name} -> {df.shape} (enc={enc}, sep={sep})")
        except Exception as e:
            print("ERROR leyendo", p, e)
    return pd.concat(tablas, axis=0, ignore_index=True) if tablas else pd.DataFrame()

def norm_key_series(s: pd.Series) -> pd.Series:
    """Normaliza valores de llave: string, sin .0 final, mayúscula, sin tildes ni separadores."""
    s = s.astype(str).str.strip().str.upper()
    s = s.str.replace(r'\.0$', '', regex=True)
    s = s.str.normalize('NFKD').str.encode('ascii','ignore').str.decode('ascii')
    s = s.str.replace(r'[\s\-\._/]', '', regex=True)
    return s

def ensure_col(df, target: str, candidates: list):
    """Renombra la primera columna existente entre candidates -> target (case/guiones/espacios tolerantes)."""
    def _norm(x): return re.sub(r'[\s_\-]+','', str(x).lower())
    if target in df.columns:
        return df
    norm_map = {_norm(c): c for c in df.columns}
    for cand in candidates:
        nc = _norm(cand)
        if nc in norm_map:
            return df.rename(columns={norm_map[nc]: target})
    # heurística adicional para codigo_unico / ruc / dni
    if target == 'codigo_unico':
        for c in df.columns:
            nc = _norm(c)
            if 'cod' in nc and 'unico' in nc:
                return df.rename(columns={c: target})
    if target == 'codigo_ruc':
        for c in df.columns:
            if 'ruc' in c.lower(): return df.rename(columns={c: target})
    if target == 'codigo_dni':
        for c in df.columns:
            if 'dni' in c.lower() or 'document' in c.lower(): return df.rename(columns={c: target})
    return df

def unify_group(df: pd.DataFrame, key: str, key_cands: list, prefer_cols: list = None, name: str = '') -> pd.DataFrame:
    """Estandariza clave, normaliza valores y deja una sola fila por clave (más completa)."""
    if df.empty:
        print(f"{name}: vacío.")
        return df
    df = ensure_col(df, key, key_cands)
    if key not in df.columns:
        raise KeyError(f"{name}: no se encontró columna clave '{key}'. Columnas: {list(df.columns)[:30]}")
    # normalizar valores de llave
    df[key] = norm_key_series(df[key])
    # score de completitud (prioriza columnas preferidas si existen)
    if prefer_cols:
        use = [c for c in prefer_cols if c in df.columns]
        if not use:
            use = [c for c in df.columns if c != key]
    else:
        use = [c for c in df.columns if c != key]
    df['_non_na_score'] = df[use].notna().sum(axis=1)
    # quedarnos con la fila más completa por clave
    df = (df.sort_values([key, '_non_na_score'], ascending=[True, False])
            .drop_duplicates(subset=[key], keep='first')
            .drop(columns=['_non_na_score']))
    print(f"{name}: {df.shape[0]} claves únicas, {df.shape[1]} columnas.")
    return df

# 1) Cargar TODOS los CSV por grupo
df_obra_raw = cargar_por_glob(CONFIG['obra_glob'])
df_emp_raw  = cargar_por_glob(CONFIG['empresa_glob'])
df_mbr_raw  = cargar_por_glob(CONFIG['miembro_glob'])
df_prio_raw = cargar_por_glob(CONFIG['prio_glob'])

print("Shapes (raw) ->",
      "Obra", df_obra_raw.shape, "| Empresa", df_emp_raw.shape,
      "| Miembro", df_mbr_raw.shape, "| Matriz", df_prio_raw.shape)

# 2) Unificar por clave en cada grupo (una fila por clave)
df_obra = unify_group(
    df_obra_raw.copy(),
    key=CONFIG['key_obra'],
    key_cands=['CODIGO_UNICO','codigo_unico','CODIGO_OBRA','IDENTIFICADOR_OBRA','id_obra','id_seace','snip'],
    # puedes priorizar columnas típicas de obra si quieres (monto/plazo/ubicación):
    prefer_cols=[c for c in ['codigo_ruc','codigo_dni','monto','valor_referencial','plazo','departamento','provincia','distrito'] if c in df_obra_raw.columns],
    name='OBRA'
)

df_emp = unify_group(
    df_emp_raw.copy(),
    key=CONFIG['key_ruc'],
    key_cands=['codigo_ruc','RUC','ruc','ruc_empresa','ruc_proveedor','COD_RUC'],
    prefer_cols=[c for c in df_emp_raw.columns if any(k in c.lower() for k in ['sancio','inhabil','arbitra','penal','multa','riesg'])],
    name='EMPRESA'
)

df_mbr = unify_group(
    df_mbr_raw.copy(),
    key=CONFIG['key_dni'],
    key_cands=['codigo_dni','DNI','dni','dni_funcionario','documento','doc_identidad','id_miembro'],
    prefer_cols=[c for c in df_mbr_raw.columns if any(k in c.lower() for k in ['sancio','respons','observa','riesg'])],
    name='MIEMBRO'
)

# Para la Matriz, priorizamos los targets
df_prio = unify_group(
    df_prio_raw.copy()
       .rename(columns={'COD_UNICO':'CODIGO_UNICO'}),  # normalizamos nombre más común
    key=CONFIG['key_obra'],  # debe quedar en 'codigo_unico'
    key_cands=['CODIGO_UNICO','codigo_unico','CODIGO_OBRA','IDENTIFICADOR_OBRA','ID_OBRA'],
    prefer_cols=[c for c in [CONFIG['prio_num'], CONFIG['prio_desc'], 'OBRA_RIESGO','OBRA_RIESGO_DESC','CODIGO_OBRA','IDENTIFICADOR_OBRA'] if c in df_prio_raw.columns],
    name='MATRIZ'
)

print("Shapes (unificados) ->",
      "Obra", df_obra.shape, "| Empresa", df_emp.shape,
      "| Miembro", df_mbr.shape, "| Matriz", df_prio.shape)


Patrón: c:\MaestriaUNI\Cursos\III-CICLO\TesisI\Solucion\Deteccion_Corrupcion\data\external\obra\DS_DASH_Obra_*.csv -> 8 archivos
Cargado: DS_DASH_Obra_1A.csv -> (326, 12) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_2A_3A.csv -> (552, 22) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_2B.csv -> (1613, 6) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_3B.csv -> (5249, 20) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_3C.csv -> (5249, 8) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_4A.csv -> (326, 9) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_4B.csv -> (326, 15) (enc=latin-1, sep=,)
Cargado: DS_DASH_Obra_5A.csv -> (634, 15) (enc=latin-1, sep=,)
Patrón: c:\MaestriaUNI\Cursos\III-CICLO\TesisI\Solucion\Deteccion_Corrupcion\data\external\empresa\DS_DASH_Empresa_*.csv -> 5 archivos
Cargado: DS_DASH_Empresa_1A.csv -> (553, 9) (enc=latin-1, sep=,)
Cargado: DS_DASH_Empresa_1B.csv -> (2148, 5) (enc=latin-1, sep=,)
Cargado: DS_DASH_Empresa_2A.csv -> (371, 11) (enc=latin-1, sep=,)
Cargado: DS_DASH_Empresa_2B.csv 

## 4) Estandarizar claves (renombrar si llegan con variantes)
Usamos las claves oficiales: `codigo_unico`, `codigo_ruc`, `codigo_dni`.

In [52]:
# ==== 4) Estandarizar claves (robusto) ====
# Objetivo:
# - Garantizar que las llaves queden con nombres estándar:
#     Obra/Matriz -> codigo_unico
#     Empresa     -> codigo_ruc
#     Miembro     -> codigo_dni
# - Normalizar los valores de llave (string, sin .0, sin separadores, mayúscula)
# - Quitar duplicados por llave
# - Dejar trazas de cobertura

import re
import pandas as pd

def _norm_name(s: str) -> str:
    return re.sub(r'[\s_\-]+', '', str(s).lower())

def ensure_col(df: pd.DataFrame, target: str, candidates: list) -> pd.DataFrame:
    """Renombra a 'target' la 1a columna que matchee (por nombre normalizado)."""
    if target in df.columns:
        return df
    norm_map = {_norm_name(c): c for c in df.columns}
    for cand in candidates:
        nc = _norm_name(cand)
        if nc in norm_map:
            return df.rename(columns={norm_map[nc]: target})
    # heurística por tokens
    for c in df.columns:
        nc = _norm_name(c)
        if target == 'codigo_unico' and ('cod' in nc and 'unico' in nc):
            return df.rename(columns={c: target})
        if target == 'codigo_ruc' and 'ruc' in nc:
            return df.rename(columns={c: target})
        if target == 'codigo_dni' and ('dni' in nc or 'document' in nc):
            return df.rename(columns={c: target})
    return df

def norm_key_series(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip().str.upper()
    s = s.str.replace(r'\.0$', '', regex=True)  # quita .0 al final
    s = s.str.normalize('NFKD').str.encode('ascii','ignore').str.decode('ascii')
    s = s.str.replace(r'[\s\-\._/]', '', regex=True)
    return s

def standardize_key(df: pd.DataFrame, key: str, key_cands: list, name: str) -> pd.DataFrame:
    df = ensure_col(df, key, key_cands)
    if key not in df.columns:
        raise KeyError(f"[{name}] No se encontró la clave '{key}'. Columnas: {list(df.columns)[:30]}")
    # normalizar valores y deduplicar
    df[key] = norm_key_series(df[key])
    before = len(df)
    df = df.drop_duplicates(subset=[key], keep='first')
    print(f"[{name}] clave='{key}' | filas: {before} -> {len(df)} (únicas) | ncols: {df.shape[1]}")
    return df

# --- Aplicar a cada grupo ---
df_obra = standardize_key(
    df_obra, key='codigo_unico',
    key_cands=['CODIGO_UNICO','COD_UNICO','CODIGO_OBRA','IDENTIFICADOR_OBRA','ID_OBRA','ID_SEACE','SNIP'],
    name='OBRA'
)

df_emp = standardize_key(
    df_emp, key='codigo_ruc',
    key_cands=['codigo_ruc','RUC','ruc','ruc_empresa','ruc_proveedor','COD_RUC'],
    name='EMPRESA'
)

df_mbr = standardize_key(
    df_mbr, key='codigo_dni',
    key_cands=['codigo_dni','DNI','dni','dni_funcionario','documento','doc_identidad','id_miembro'],
    name='MIEMBRO'
)

# La matriz se alinea a la llave de OBRA
df_prio = standardize_key(
    df_prio.rename(columns={'COD_UNICO':'CODIGO_UNICO'}),  # por si vino así
    key='codigo_unico',
    key_cands=['CODIGO_UNICO','COD_UNICO','CODIGO_OBRA','IDENTIFICADOR_OBRA','ID_OBRA'],
    name='MATRIZ'
)

# --- Diagnóstico rápido de intersección de llaves (previo al merge de la sección 7) ---
obra_keys   = set(df_obra['codigo_unico'])
prio_keys   = set(df_prio['codigo_unico']) if 'codigo_unico' in df_prio.columns else set()
emp_keys    = set(df_emp['codigo_ruc'])    if 'codigo_ruc' in df_emp.columns else set()
mbr_keys    = set(df_mbr['codigo_dni'])    if 'codigo_dni' in df_mbr.columns else set()

print(f"Intersección Obra∩Matriz: {len(obra_keys & prio_keys)} / {len(obra_keys)}")
print(f"Claves únicas -> Obra:{len(obra_keys)}  Matriz:{len(prio_keys)}  Empresa:{len(emp_keys)}  Miembro:{len(mbr_keys)}")



[OBRA] clave='codigo_unico' | filas: 171 -> 171 (únicas) | ncols: 55
[EMPRESA] clave='codigo_ruc' | filas: 372 -> 372 (únicas) | ncols: 25
[MIEMBRO] clave='codigo_dni' | filas: 740 -> 740 (únicas) | ncols: 16
[MATRIZ] clave='codigo_unico' | filas: 822 -> 822 (únicas) | ncols: 26
Intersección Obra∩Matriz: 170 / 171
Claves únicas -> Obra:171  Matriz:822  Empresa:372  Miembro:740


## 5) Agregados por Empresa (RUC) y Miembro (DNI)

In [55]:
# ==== 5) Agregados por Empresa (RUC) y Miembro (DNI) ====
# Requiere: df_emp (clave codigo_ruc) y df_mbr (clave codigo_dni) ya estandarizados (Secc. 3 y 4)

import numpy as np
import pandas as pd

def to_numeric_safe(df, cols_like):
    """Conserva sólo columnas que matchean 'cols_like' y las convierte a numérico.
       Mapea SI/NO/True/False → 1/0 cuando aplique."""
    cols = [c for c in df.columns if any(k in c.lower() for k in cols_like)]
    if not cols:
        return pd.DataFrame(index=df.index)
    tmp = df[cols].copy()

    for c in tmp.columns:
        if tmp[c].dtype == object:
            s = tmp[c].astype(str).str.strip().str.lower()
            mask = s.isin(['si','sí','true','t','1','no','false','f','0'])
            if mask.any():
                s = s.replace({'si':1,'sí':1,'true':1,'t':1,'1':1,
                               'no':0,'false':0,'f':0,'0':0})
                tmp[c] = pd.to_numeric(s, errors='coerce')
            else:
                tmp[c] = pd.to_numeric(tmp[c], errors='coerce')
        else:
            tmp[c] = pd.to_numeric(tmp[c], errors='coerce')

    keep = [c for c in tmp.columns if tmp[c].notna().any()]
    return tmp[keep]


def aggregate_by_key(df, key, risk_like, prefix):
    """sum/mean/max por clave en columnas numéricas detectadas por 'risk_like'."""
    if key not in df.columns or df.empty:
        return pd.DataFrame()
    numdf = to_numeric_safe(df, risk_like)
    if numdf.empty:
        return pd.DataFrame()

    agg = (
        pd.concat([df[[key]], numdf], axis=1)
          .groupby(key)
          .agg(['sum','mean','max'])
    )
    agg.columns = [f"{prefix}__{col}_{stat}" for col, stat in agg.columns]
    agg = agg.reset_index()

    # Añadir conteo de registros por clave
    counts = df.groupby(key).size().rename(f"{prefix}__n_registros").reset_index()
    agg = agg.merge(counts, on=key, how='left')
    return agg

# --- Normalización previa para Empresa: estados -> 0/1 y conteos numéricos
import re
def estado_to_flag(s):
    if pd.isna(s): 
        return np.nan
    x = str(s).strip().upper()
    if 'NORMAL' in x:
        return 0
    # cualquier señal de sanción/inhabilitación/suspensión/impedimento/multa
    if re.search(r'(SANCION|INHABIL|SUSPEN|IMPED|MULTA)', x):
        return 1
    return np.nan  # si no matchea nada, lo dejamos como NaN

# columnas categóricas de estado (ajusta si ves otras)
estado_cols = [
    'SANCIONADAS_TCE','INHABILITADAS_PJ','SANCIONADAS_RNP','INHABILITADAS_RNP'
]
estado_cols = [c for c in estado_cols if c in df_emp.columns]

for c in estado_cols:
    df_emp[c + '_FLAG'] = df_emp[c].apply(estado_to_flag)

# columnas de conteo (asegurar numérico)
count_cols = ['NUMERO_SANCIONES_TCE','NUMERO_SANCIONES_RNP']
for c in [cc for cc in count_cols if c in df_emp.columns]:
    df_emp[c] = pd.to_numeric(df_emp[c], errors='coerce').fillna(0)

# (opcional) muestra rápida
cols_show = ['codigo_ruc'] + [c+'_FLAG' for c in estado_cols] + [c for c in count_cols if c in df_emp.columns]
display(df_emp[cols_show].head(8))


# --- Empresa (RUC) ---
EMP_LIKE = ['riesg','sancio','inhabil','arbitra','penal','multa','mora']
df_emp_agg = aggregate_by_key(df_emp, 'codigo_ruc', EMP_LIKE, prefix='empresa')
print('Empresa agregados:', df_emp_agg.shape, '| cols:', len([c for c in df_emp_agg.columns if c.startswith('empresa__')]))

# --- Miembro (DNI) ---
MBR_LIKE = ['riesg','sancio','respons','observa','inhabil']
df_mbr_agg = aggregate_by_key(df_mbr, 'codigo_dni', MBR_LIKE, prefix='miembro')
print('Miembro agregados:', df_mbr_agg.shape, '| cols:', len([c for c in df_mbr_agg.columns if c.startswith('miembro__')]))



,codigo_ruc,SANCIONADAS_TCE_FLAG,INHABILITADAS_PJ_FLAG,SANCIONADAS_RNP_FLAG,INHABILITADAS_RNP_FLAG,NUMERO_SANCIONES_TCE,NUMERO_SANCIONES_RNP
172,10001046191,0.0,0.0,0.0,0.0,0.0,0.0
381,10011010534,0.0,0.0,0.0,0.0,0.0,0.0
274,10011217104,0.0,0.0,0.0,0.0,0.0,0.0
267,10011600447,0.0,0.0,0.0,0.0,0.0,0.0
182,10020343112,0.0,0.0,0.0,0.0,0.0,0.0
463,10021511116,0.0,0.0,0.0,0.0,0.0,0.0
427,10024475005,0.0,0.0,0.0,0.0,0.0,0.0
139,10031271369,0.0,0.0,0.0,0.0,0.0,0.0


Empresa agregados: (372, 20) | cols: 19
Miembro agregados: (740, 11) | cols: 10


In [56]:
print("Tiene clave codigo_ruc en df_emp?:", 'codigo_ruc' in df_emp.columns)
print("Filas no nulas en codigo_ruc:", df_emp['codigo_ruc'].notna().sum() if 'codigo_ruc' in df_emp.columns else 0)

EMP_LIKE = ['riesg','sancio','inhabil','arbitra','penal','multa','mora']
emp_match_cols = [c for c in df_emp.columns if any(k in c.lower() for k in EMP_LIKE)]
print("Columnas empresa que matchean patrones:", emp_match_cols[:30], " (total:", len(emp_match_cols), ")")

# Mira 5 filas de esas columnas
if emp_match_cols:
    display(df_emp[['codigo_ruc'] + emp_match_cols].head())


Tiene clave codigo_ruc en df_emp?: True
Filas no nulas en codigo_ruc: 372
Columnas empresa que matchean patrones: ['SANCIONADAS_TCE', 'INHABILITADAS_PJ', 'SANCIONADAS_RNP', 'INHABILITADAS_RNP', 'NUMERO_SANCIONES_TCE', 'NUMERO_SANCIONES_RNP', 'SANCIONADAS_TCE_FLAG', 'INHABILITADAS_PJ_FLAG', 'SANCIONADAS_RNP_FLAG', 'INHABILITADAS_RNP_FLAG']  (total: 10 )


,codigo_ruc,SANCIONADAS_TCE,INHABILITADAS_PJ,SANCIONADAS_RNP,INHABILITADAS_RNP,NUMERO_SANCIONES_TCE,NUMERO_SANCIONES_RNP,SANCIONADAS_TCE_FLAG,INHABILITADAS_PJ_FLAG,SANCIONADAS_RNP_FLAG,INHABILITADAS_RNP_FLAG
172,10001046191,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,0.0,0.0,0.0,0.0,0.0,0.0
381,10011010534,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,0.0,0.0,0.0,0.0,0.0,0.0
274,10011217104,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,0.0,0.0,0.0,0.0,0.0,0.0
267,10011600447,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,0.0,0.0,0.0,0.0,0.0,0.0
182,10020343112,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,EN SITUACION NORMAL,0.0,0.0,0.0,0.0,0.0,0.0


## 6) Merge: enriquecer Obra con agregados por RUC y DNI

In [59]:
# ==== 6) Merge: enriquecer Obra con agregados por RUC y DNI (robusto) ====
df_maestro = df_obra.copy()

# Detectar columnas RUC/DNI en Obra y estandarizarlas localmente
RUC_OBRA_CANDS = ['codigo_ruc','ruc','ruc_empresa','ruc_proveedor']
DNI_OBRA_CANDS = ['codigo_dni','dni','dni_funcionario','documento','doc_identidad','id_miembro']

def pick_col(df, cands):
    for c in cands:
        if c in df.columns: return c
    return None

col_ruc_obra = pick_col(df_maestro, RUC_OBRA_CANDS)
col_dni_obra = pick_col(df_maestro, DNI_OBRA_CANDS)

if col_ruc_obra:
    df_maestro.rename(columns={col_ruc_obra: 'codigo_ruc_obra'}, inplace=True)
if col_dni_obra:
    df_maestro.rename(columns={col_dni_obra: 'codigo_dni_obra'}, inplace=True)

def norm_key(s):
    return (s.astype(str).str.strip().str.upper()
              .str.replace(r'\.0$','',regex=True)
              .str.normalize('NFKD').str.encode('ascii','ignore').str.decode('ascii')
              .str.replace(r'[\s\-\._/]','',regex=True))

# Normalizar llaves en todos
if 'codigo_ruc_obra' in df_maestro.columns:
    df_maestro['codigo_ruc_obra'] = norm_key(df_maestro['codigo_ruc_obra'])
if 'codigo_dni_obra' in df_maestro.columns:
    df_maestro['codigo_dni_obra'] = norm_key(df_maestro['codigo_dni_obra'])

if not df_emp_agg.empty:
    df_emp_agg['codigo_ruc'] = norm_key(df_emp_agg['codigo_ruc'])
    df_emp_agg = df_emp_agg.drop_duplicates('codigo_ruc')
if not df_mbr_agg.empty:
    df_mbr_agg['codigo_dni'] = norm_key(df_mbr_agg['codigo_dni'])
    df_mbr_agg = df_mbr_agg.drop_duplicates('codigo_dni')

# Merge + cobertura
if not df_emp_agg.empty and 'codigo_ruc_obra' in df_maestro.columns:
    before = len(df_maestro)
    df_maestro = df_maestro.merge(df_emp_agg, left_on='codigo_ruc_obra', right_on='codigo_ruc', how='left')
    matched_emp = df_maestro['codigo_ruc'].notna().sum()
    print(f'Empresa: {matched_emp}/{before} obras con match por RUC')

if not df_mbr_agg.empty and 'codigo_dni_obra' in df_maestro.columns:
    before = len(df_maestro)
    df_maestro = df_maestro.merge(df_mbr_agg, left_on='codigo_dni_obra', right_on='codigo_dni', how='left')
    matched_mbr = df_maestro['codigo_dni'].notna().sum()
    print(f'Miembro : {matched_mbr}/{before} obras con match por DNI')

df_maestro.shape


(171, 55)

## 7) Etiquetado desde **Matriz de Priorización** (y fallback por reglas)
- Si existen `PROYECTO_RIESGO` y/o `PROYECTO_RIESGO_DESC` en la matriz, se mapean a `y_riesgo`.
- Obras sin etiqueta en matriz se completan con **reglas** (penalidades/arbitrajes/adicionales/ampliaciones/rescisión/sanciones/observaciones).

In [71]:
# ==== MATRIZ: construir tabla de llaves + TODOS los candidatos de etiqueta ====
import pandas as pd, numpy as np, re

def norm_key(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip().str.upper()
    s = s.str.replace(r'\.0$','', regex=True)
    s = s.str.normalize('NFKD').str.encode('ascii','ignore').str.decode('ascii')
    s = s.str.replace(r'[\s\-\._/]', '', regex=True)
    return s

def coalesce_cols(df, cols):
    out = pd.Series(pd.NA, index=df.index, dtype="object")
    for c in [c for c in cols if c in df.columns]:
        out = out.where(out.notna(), df[c])
    return out

# 1) JOIN_KEY en obra y matriz (coalesce de IDs más comunes)
obra_ids   = ['CODIGO_UNICO','codigo_unico','CODIGO_OBRA','IDENTIFICADOR_OBRA','ID_OBRA']
matriz_ids = ['COD_UNICO','CODIGO_UNICO','codigo_unico','CODIGO_OBRA','IDENTIFICADOR_OBRA','ID_OBRA']

df_maestro['JOIN_KEY'] = norm_key(coalesce_cols(df_maestro, obra_ids))
df_prio['_JOIN_KEY']   = norm_key(coalesce_cols(df_prio, matriz_ids))

# 2) Elegir columnas de etiqueta candidatas
label_num_cols  = [c for c in ['PROYECTO_RIESGO','OBRA_RIESGO'] if c in df_prio.columns]
label_desc_cols = [c for c in ['PROYECTO_RIESGO_DESC','OBRA_RIESGO_DESC'] if c in df_prio.columns]

keep_cols = ['_JOIN_KEY'] + label_num_cols + label_desc_cols
df_prio_keys = df_prio[keep_cols].dropna(subset=['_JOIN_KEY']).copy()

# 3) Para cada JOIN_KEY, quedarnos con la fila que tenga más etiquetas no nulas
score_cols = label_num_cols + label_desc_cols
if score_cols:
    df_prio_keys['_score'] = df_prio_keys[score_cols].notna().sum(axis=1)
    df_prio_keys = (df_prio_keys
                    .sort_values(['_JOIN_KEY','_score'], ascending=[True, False])
                    .drop_duplicates('_JOIN_KEY', keep='first')
                    .drop(columns=['_score']))

df_prio_keys = df_prio_keys.rename(columns={'_JOIN_KEY':'JOIN_KEY'})

# 4) Diagnóstico de cobertura antes del merge
inter = set(df_maestro['JOIN_KEY'].dropna().unique()) & set(df_prio_keys['JOIN_KEY'].dropna().unique())
print("Intersección de JOIN_KEY:", len(inter))
print("Cobertura etiqueta en intersección:")
if inter:
    sub = df_prio_keys[df_prio_keys['JOIN_KEY'].isin(inter)]
    print(" - num no nulos:", sub[label_num_cols].notna().sum().to_dict() if label_num_cols else {})
    print(" - desc no nulos:", sub[label_desc_cols].notna().sum().to_dict() if label_desc_cols else {})

# 5) Evitar colisiones con columnas ya presentes
cols_right = set(df_prio_keys.columns) - {'JOIN_KEY'}
cols_overlap = list(set(df_maestro.columns) & cols_right)
if cols_overlap:
    print("Eliminando del izquierdo por colisión:", cols_overlap)
    df_maestro = df_maestro.drop(columns=cols_overlap, errors='ignore')
to_drop_prev = [c for c in df_maestro.columns if c.endswith('_x') or c.endswith('_y')]
if to_drop_prev:
    print("Limpiando restos de merges previos:", to_drop_prev)
    df_maestro = df_maestro.drop(columns=to_drop_prev, errors='ignore')

# 6) MERGE (m:1)
before = len(df_maestro)
df_maestro = df_maestro.merge(df_prio_keys, on='JOIN_KEY', how='left', validate='m:1')

# 7) Construcción de etiqueta combinada
#   - y_num_src: primero PROYECTO_RIESGO, si NaN usar OBRA_RIESGO (si existen)
y_num_src = None
if label_num_cols:
    if 'PROYECTO_RIESGO' in label_num_cols and 'OBRA_RIESGO' in label_num_cols:
        y_num_src = df_maestro['PROYECTO_RIESGO'].copy()
        y_num_src = y_num_src.where(y_num_src.notna(), df_maestro['OBRA_RIESGO'])
    else:
        y_num_src = df_maestro[label_num_cols[0]].copy()

#   - y_desc_src: primero PROYECTO_RIESGO_DESC, si NaN usar OBRA_RIESGO_DESC (si existen)
y_desc_src = None
if label_desc_cols:
    if 'PROYECTO_RIESGO_DESC' in label_desc_cols and 'OBRA_RIESGO_DESC' in label_desc_cols:
        y_desc_src = df_maestro['PROYECTO_RIESGO_DESC'].copy()
        y_desc_src = y_desc_src.where(y_desc_src.notna(), df_maestro['OBRA_RIESGO_DESC'])
    else:
        y_desc_src = df_maestro[label_desc_cols[0]].copy()

# 8) y_riesgo: preferir numérico; si no hay, usar descriptivo
y = None
if y_num_src is not None and y_num_src.notna().any():
    y = (pd.to_numeric(y_num_src, errors='coerce').fillna(0) > 0).astype(int)
elif y_desc_src is not None and y_desc_src.notna().any():
    y = (y_desc_src.astype(str).str.lower().str.strip()
         .isin(['alto','medio','riesgoso','1','si','sí','true'])).astype(int)

df_maestro['y_riesgo'] = y if y is not None else 0

# 9) Reporte final
matched_num  = int(y_num_src.notna().sum()) if y_num_src is not None else 0
matched_desc = int(y_desc_src.notna().sum()) if y_desc_src is not None else 0
print(f"Etiquetas numéricas traídas: {matched_num} | descriptivas traídas: {matched_desc}")
print('Distribución final y_riesgo:')
print(df_maestro['y_riesgo'].value_counts(dropna=False))


Intersección de JOIN_KEY: 170
Cobertura etiqueta en intersección:
 - num no nulos: {'PROYECTO_RIESGO': 0, 'OBRA_RIESGO': 170}
 - desc no nulos: {'PROYECTO_RIESGO_DESC': 0, 'OBRA_RIESGO_DESC': 170}
Eliminando del izquierdo por colisión: ['PROYECTO_RIESGO', 'PROYECTO_RIESGO_DESC']
Etiquetas numéricas traídas: 170 | descriptivas traídas: 170
Distribución final y_riesgo:
y_riesgo
1    153
0     18
Name: count, dtype: int64


In [73]:
print("Únicas JOIN_KEY Obra:", df_maestro['JOIN_KEY'].nunique())
print("Únicas JOIN_KEY Matriz:", df_prio['_JOIN_KEY'].nunique() if '_JOIN_KEY' in df_prio.columns else 'N/A')
print("Ejemplos Obra:", df_maestro['JOIN_KEY'].dropna().unique()[:5])
print("Ejemplos Matriz:", df_prio['_JOIN_KEY'].dropna().unique()[:5])


Únicas JOIN_KEY Obra: 171
Únicas JOIN_KEY Matriz: 822
Ejemplos Obra: ['2002060' '2002210' '2015918' '2026767' '2027711']
Ejemplos Matriz: ['2002060' '2002210' '2002604' '2015918' '2016958']


## 8) Exportar parquet y chequeos rápidos

In [76]:
import re
import numpy as np
import pandas as pd

# 1) Quita columnas auxiliares y residuos de merges
aux_patterns = [
    r'_x$', r'_y$', r'_N$', r'_D$', r'^_key$', r'^JOIN_KEY$',
    r'^codigo_ruc_obra$', r'^codigo_dni_obra$'
]
drop_aux = [c for c in df_maestro.columns if any(re.search(p, c) for p in aux_patterns)]
if drop_aux:
    print("Eliminando auxiliares:", len(drop_aux))
    df_maestro = df_maestro.drop(columns=drop_aux, errors='ignore')

# 2) Coalesce de columnas DUPLICADAS por nombre (conserva 1 columna unificada)
def coalesce_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = df.columns
    # agrupar índices de columnas por nombre
    groups = cols.to_series().groupby(cols).groups
    for name, idxs in groups.items():
        if len(idxs) > 1:
            # combinar de izquierda→derecha: primer no-nulo
            block = df.iloc[:, list(idxs)]
            combined = block.bfill(axis=1).iloc[:, 0]
            # borrar todas las repetidas y dejar una sola con el valor combinado
            df = df.drop(columns=block.columns)
            df[name] = combined
    return df

print("Columnas antes de coalesce:", df_maestro.shape[1])
df_maestro = coalesce_duplicate_columns(df_maestro)
print("Columnas después de coalesce:", df_maestro.shape[1])

# 3) Asegurar que 'y_riesgo' exista UNA vez y sea int {0,1}
if 'y_riesgo' in df_maestro.columns:
    df_maestro['y_riesgo'] = pd.to_numeric(df_maestro['y_riesgo'], errors='coerce').fillna(0).astype(int)
else:
    raise ValueError("No se encontró la columna 'y_riesgo' para exportar.")

# 4) Preparar lista final de features (evita fugas/IDs)
cols_leak = [
    'PROYECTO_RIESGO','PROYECTO_RIESGO_DESC','OBRA_RIESGO','OBRA_RIESGO_DESC',
    'CODIGO_UNICO','codigo_unico','CODIGO_OBRA','IDENTIFICADOR_OBRA'
]
features = [c for c in df_maestro.columns if c not in cols_leak + ['y_riesgo']]

# 5) Exportar parquet
df_out = df_maestro[features + ['y_riesgo']].copy()
df_out.to_parquet(CONFIG['output_parquet'], index=False)
print('✅ Guardado:', CONFIG['output_parquet'], '->', df_out.shape)
print('Target dist:\n', df_out['y_riesgo'].value_counts())


Eliminando auxiliares: 5
Columnas antes de coalesce: 60
Columnas después de coalesce: 60
✅ Guardado: c:\MaestriaUNI\Cursos\III-CICLO\TesisI\Solucion\Deteccion_Corrupcion\data\processed\dataset_obras.parquet -> (171, 53)
Target dist:
 y_riesgo
1    153
0     18
Name: count, dtype: int64
